In [2]:
# all this is same as before
 
import os
import kagglehub
import pandas as pd
import numpy as np
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
import spacy
from sklearn.feature_extraction.text import CountVectorizer
import fasttext.util as fasttext_util
import fasttext
from sklearn.metrics import f1_score
from collections import Counter
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
import tqdm
import pickle
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
import evaluate
from sklearn.metrics import f1_score, accuracy_score

# For Google Colab (if needed)
# from google.colab import drive

# specify custom functions --------------------------------------------------------------
def custom_tokenizer(text):
    tokenized_text = nlp(text)
    return [tok.text for tok in tokenized_text]

def embedding_mapping_fasttext(vocabulary, pre_trained_embeddings):
    vocab_size = len(vocabulary)
    embedding_dim = pre_trained_embeddings.get_dimension()
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for idx, word in enumerate(vocabulary):
        embedding_matrix[idx] = pre_trained_embeddings.get_word_vector(word)
    return embedding_matrix

# download pretrained embeddings --------------------------------------------------------
# for local
#fasttext_util.download_model('en', if_exists='ignore')

# for Gdrive
drive.mount('/content/drive')
model_path = "/content/drive/MyDrive/cc.en.300.bin"
ft = fasttext.load_model(model_path)




/Users/henrybaker/miniconda3/envs/nlp_1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# reading climate df
input_path_climate = "/Users/henrybaker/.cache/kagglehub/datasets/die9origephit/climate-change-tweets/versions/1/Climate change_2022-1-17_2022-7-19.csv"

output_path_climate = "/Users/henrybaker/Documents/repositories/NLP/nlp_project/data/climate-change-tweets.csv"

df_climate = pd.read_csv(input_path_climate)
print(f"Loading dataset from '{input_path_climate}'...")
df_climate.head()

Loading dataset from '/Users/henrybaker/.cache/kagglehub/datasets/die9origephit/climate-change-tweets/versions/1/Climate change_2022-1-17_2022-7-19.csv'...


,UserScreenName,UserName,Timestamp,Text,Embedded_text,Emojis,Comments,Likes,Retweets,Image link,Tweet URL
0,Lauren Boebert,@laurenboebert,2022-01-17T23:32:38.000Z,Lauren Boebert\n@laurenboebert\n·\nJan 18,The only solution I’ve ever heard the Left pro...,NaN,"1,683","2,259",11.7K,[],https://twitter.com/laurenboebert/status/14832...
1,Catherine,@catherine___c,2022-01-17T22:54:02.000Z,Catherine\n@catherine___c\n·\nJan 17,Climate change doesn’t cause volcanic eruption...,NaN,158,64,762,[],https://twitter.com/catherine___c/status/14832...
2,king Keith,@KaConfessor,2022-01-17T23:51:41.000Z,king Keith\n@KaConfessor\n·\nJan 18,Vaccinated tennis ball boy collapses in the te...,NaN,24,118,159,['https://pbs.twimg.com/ext_tw_video_thumb/148...,https://twitter.com/KaConfessor/status/1483225...
3,PETRIFIED CLIMATE PARENT,@climate_parent,2022-01-17T21:42:04.000Z,PETRIFIED CLIMATE PARENT\n@climate_parent\n·\n...,North America has experienced an average winte...,NaN,15,50,158,[],https://twitter.com/climate_parent/status/1483...
4,Thomas Speight,@Thomas_Sp8,2022-01-17T21:10:40.000Z,Thomas Speight\n@Thomas_Sp8\n·\nJan 17,They're gonna do the same with Climate Change ...,🅾,4,24,127,['https://pbs.twimg.com/profile_images/1544171...,https://twitter.com/Thomas_Sp8/status/14831850...


In [4]:
df_climate_inference = df_climate[['Embedded_text']].rename(columns={'Embedded_text': 'text'})
df_climate_inference['label'] = None

print(f"Train shape {df_climate_inference.shape} \n")
df_climate_inference.head()




Train shape (9050, 2) 



,text,label
0,The only solution I’ve ever heard the Left pro...,None
1,Climate change doesn’t cause volcanic eruption...,None
2,Vaccinated tennis ball boy collapses in the te...,None
3,North America has experienced an average winte...,None
4,They're gonna do the same with Climate Change ...,None


NB: use the **same** tokenizer (with the same learned vocabulary) when applying my trained model to new data. Typically, we train the tokenizer **only** on the training set (misinformation tweets) to avoid data leakage. Then, at inference time, you apply that exact tokenizer to the climate tweets.

---

## Why Use the Same Trained Tokenizer?

1. **Consistent Feature Representation:**  
   Your model has learned to map _specific token indices_ (or token n-grams) to output classes. If you change the tokenizer (e.g., by retraining it on different data or adding new vocabulary), the numeric indices won’t match the same words your model learned during training. This mismatch can degrade performance or completely break your inference pipeline.

2. **Avoid Data Leakage:**  
   Training the tokenizer on any unlabeled “future” data (the climate tweets) can inadvertently leak information about that new dataset into your model pipeline, which is generally considered poor practice. Typically, the training set should define the vocabulary for your model.

3. **Practical Workflows in NLP:**  
   In real-world scenarios, your training pipeline is frozen at some point (with its tokenizer, label encoder, etc.). When you deploy your model to predict on new data, you must replicate _exactly_ the same text-preprocessing steps.

---

## What if the Climate Tweets Contain New or Domain-Specific Vocabulary?

- **Option 1: Accept Out-of-Vocabulary Words**  
  If the climate dataset uses words that weren’t in the original misinformation tweets vocabulary, your tokenizer will treat them as out-of-vocabulary (OOV) or handle them in some fallback way (e.g., ignoring them or assigning them to a generic “unknown” token). This is normal in most NLP pipelines: if the new text has unseen terms, the model can’t magically handle them unless the meaning can be derived from context or subword embeddings.

- **Option 2: Retrain or Extend the Vocabulary**  
  If your new dataset is large, domain-specific, and you want the model to learn from it, you could retrain or fine-tune the entire pipeline (including the tokenizer) on a combined dataset (misinformation + climate). However, that means retraining (or at least fine-tuning) the model from scratch. This is more advanced and depends on your use case:
  1. **Combine both datasets** into a single corpus for tokenizer training.  
  2. Tokenize both datasets with the newly trained tokenizer.  
  3. Retrain the classifier using the combined or partial data approach.

  But if your goal is purely inference (i.e., you are not re-training the model, just applying it to climate tweets), _stick to the trained tokenizer_.

In [ ]:
# Check if the pickle files already exist
# local
climate_tokens_file = 'climate_tokens.pkl'


# for Hertie GPU:
# climate_tokens_file = '/workspace/workspace/cache/climate_tokens.pkl'


if os.path.exists(climate_tokens_file):
    print("Tokenized text pkl files found: loading data...")
    # Load the pre-saved tokenized data
    with open(climate_tokens_file, 'rb') as f:
        climate_tokens = pickle.load(f)
else:
    print("Pickle files not found. Running tokenization...")

    print("Loading spaCy model...")
    nlp = spacy.load("en_core_web_sm", disable=["tok2vec", "tagger", "parser", "ner", "lemmatizer", "attribute_ruler"])

    print("Tokenizing Train Data in Batches...")
    climate_tokens = batch_tokenize(df_climate_inference, misinfo_tokenizer_analyzer)

    # Save tokenized train and test data
    with open(climate_tokens_file, 'wb') as f:
        pickle.dump(climate_tokens, f)


In [ ]:
# STEP 1: INPUT PIPELINE ================================================================

# vocabulary indexing -------------------------------------------------------------------
print ("vocab indexing")

def vocab_mapping(tokenized_text):
    token_counts = Counter()
    for text in tokenized_text:
        token_counts.update(text)
    special_tokens = ["<pad>", "<unk>"]
    vocab_tokens = special_tokens + [token for token, freq in token_counts.most_common()]
    vocab = {token: idx for idx, token in enumerate(vocab_tokens)}
    return vocab

vocab_idx = vocab_mapping(tokenized_text=misinfo_train_tokens)

# create data loaders -------------------------------------------------------------------

print("creating data loaders")

def collate_fn(data):
    text_list, label_list = [], []
    for _text, _label in data:
        # integer encoding with truncation
        processed_text = torch.tensor([vocab_idx[token] for token in _text][:max_seq_length],
                                      dtype=torch.int64)
        text_list.append(processed_text)
        label_list.append(_label)
    label_list = torch.tensor(label_list)
    # padding
    padded_text_list = nn.utils.rnn.pad_sequence(text_list,
                                                 batch_first=True,
                                                 padding_value=0)
    return padded_text_list, label_list

max_seq_length = 300
batch_size = 32

train_dl = DataLoader(dataset=list(zip(misinfo_train_tokens,
                                         df_misinfo_train["label"])), # was meant to be balanced
                        batch_size=32, shuffle=True, collate_fn=collate_fn)
test_dl = DataLoader(dataset=list(zip(misinfo_test_tokens,
                                         df_misinfo_test["label"])),
                        batch_size=32, shuffle=False, collate_fn=collate_fn)

print("created data loaders")

# map pretrained fasttext embeddings to vocabulary indices ------------------------------

# Define the file path for the pickle file
# for local:
pickle_file_path = "mapped_pretrained_embeddings.pkl"
# for Hertie GPU:
#pickle_file_path = "/workspace/workspace/mapped_pretrained_embeddings.pkl"

# Check if the pickle file already exists
if os.path.exists(pickle_file_path):
    # If the file exists, load it from the pickle file
    with open(pickle_file_path, 'rb') as f:
        embedding_tensor = pickle.load(f)
    print(f"Emebddings pre-exists: loaded embeddings from {pickle_file_path}. Shape: {embedding_tensor.shape}")
else:
    # If the file does not exist, proceed with creating the embeddings and save them
    # Load pre-trained FastText model
    print("Embeddings do not pre-exist: mapping pretrained fasttext embeddings to vocabulary indices")

    # Map pretrained FastText embeddings to vocabulary indices
    mapped_pretrained_embeddings = embedding_mapping_fasttext(vocabulary=vocab_idx,
                                                              pre_trained_embeddings=ft)

    # Convert mapped embeddings to a tensor
    embedding_tensor = torch.FloatTensor(mapped_pretrained_embeddings)

    # Save the embeddings to a pickle file
    with open(pickle_file_path, 'wb') as f:
        pickle.dump(embedding_tensor, f)
    print(f"Saved embeddings to {pickle_file_path}. Shape: {embedding_tensor.shape}")